# Gated Convoluntional Neural Networks

<div style="display: flex; align-items: center;">
    <img src="../imgs/GatedCNN.jpg" alt="Your Image" width="300" style="margin-right: 20px;">
    <div>
        <p>Gated Convolutional Neural Networks (Gated CNNs) are an advanced architectural variant of traditional Convolutional Neural Networks (CNNs) that incorporate gating mechanisms to regulate the flow of information through the network. These gating units act like learnable switches that can open or close, allowing certain features to pass while suppressing others, thus enabling the network to focus on relevant information.</p>
        <p>The core of a Gated CNN is its gating mechanism, which typically uses sigmoid functions to generate gate signals. These signals produce values between 0 and 1, determining how much of the input should be let through.</p>
        <p>Gated CNNs often employ depthwise separable convolutions for the token mixer, which are computationally efficient. This involves first performing a depthwise convolution that acts on each input channel separately, followed by a pointwise convolution that combines the outputs.</p>
        <p>In addition, this chapter will introduce a more challenging dataset: CIFAR10/CIFAR100, and attempt to challenge them.</p>
    </div>
</div>

## Gated CNN
Before building model.Let's see the parameters of the new layer:<br>
**nn.LayerNorm**:<br>
`normalized_shape`: The shape of the normalization, this could be a single integer N or a tuple of integers representing the last N dimensions to normalize over.<br>
`eps`: A small constant added for numerical stability. This prevents the division by zero error during the normalization process.<br>
`elementwise_affine`: A boolean value which, when set to True, applies a learnable scaling and shifting transformation after normalization.

In [1]:
import torch
import torch.nn as nn
from functools import partial

class GatedCNN(nn.Module):
    
    def __init__(self, input_channels, output_size):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.BatchNorm2d(64)
        )
        self.downsample = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        self.blocks = nn.Sequential(
            GatedCNNBlock(128, kernel_size=3),
            GatedCNNBlock(128, kernel_size=3),
            GatedCNNBlock(128, kernel_size=3),
            GatedCNNBlock(128, kernel_size=3),
            GatedCNNBlock(128, kernel_size=3)
        )
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, output_size)

    def forward(self, x):
        x = self.stem(x)
        x = self.downsample(x)
        x = self.blocks(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

class GatedCNNBlock(nn.Module):

    def __init__(self, dim, expension_ratio=8/3, kernel_size=7, conv_ratio=1.0,
                 norm_layer=partial(nn.LayerNorm, eps=1e-6), 
                 act_layer=nn.GELU,
                 drop_path=0.,
                 **kwargs):
        super().__init__()
        hidden = int(expension_ratio * dim)
        conv_channels = int(conv_ratio * dim)
        self.split_indices = (hidden, hidden - conv_channels, conv_channels)
        
        self.norm = norm_layer(dim)
        self.fc1 = nn.Linear(dim, hidden * 2)
        self.act = act_layer()
        self.conv = nn.Conv2d(conv_channels, conv_channels, kernel_size=kernel_size, padding=kernel_size//2, groups=conv_channels)
        self.fc2 = nn.Linear(hidden, dim)

    def forward(self, x):
        shortcut = x  # [B, C, H, W]
        x = x.permute(0, 2, 3, 1)  # [B, C, H, W] -> [B, H, W, C]
        x = self.norm(x)
        g, i, c = torch.split(self.fc1(x), self.split_indices, dim=-1)
        c = c.permute(0, 3, 1, 2)  # [B, H, W, C] -> [B, C, H, W]
        c = self.conv(c)
        c = c.permute(0, 2, 3, 1)  # [B, C, H, W] -> [B, H, W, C]
        x = self.fc2(self.act(g) * torch.cat((i, c), dim=-1))
        x = x.permute(0, 3, 1, 2)  # [B, H, W, C] -> [B, C, H, W]
        return x + shortcut

## CIFAR10/100 dataset
The CIFAR dataset is a widely used benchmark in the field of machine learning and computer vision, particularly for image classification tasks. Here are some of its key characteristics:

### CIFAR Dataset Characteristics
**Two Variants**: CIFAR comes in two versions - CIFAR-10 and CIFAR-100:<br>
`CIFAR-10`: Contains 60,000 32x32 color images in 10 classes, with 6,000 images per class. It is split into 50,000 training images and 10,000 test images.
`CIFAR-100`: Similar to `CIFAR-10` but has 100 classes with 600 images per class. Each class in CIFAR-100 is divided into a "fine" and "coarse" label structure.

- Image Size: All images are 32x32 pixels with 3 color channels (RGB), making the dataset relatively small in terms of image resolution.

- Diversity: The dataset includes a wide variety of images in each class, making it more challenging than simpler datasets like MNIST.

**Comparison with MNIST Complexity**:<br>
`MNIST`: Consists of grayscale images of handwritten digits (0-9), with each image being 28x28 pixels. The dataset is relatively simple with less variation in the data.<br>
`CIFAR`: Contains color images with greater variability in terms of objects, backgrounds, and lighting conditions, making classification more challenging.<br>
**Number of Classes**:<br>
MNIST: 10 classes (digits 0-9).<br>
CIFAR-10: 10 classes (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck).<br>
CIFAR-100: 100 classes, adding significantly more complexity compared to both MNIST and CIFAR-10.<br>

**Image Resolution and Color Channels**:<br>
MNIST: 28x28 pixel grayscale images (1 channel).<br>
CIFAR: 32x32 pixel color images (3 channels), requiring models to handle more data and more complex features.

In [2]:
import sys
sys.path.append('../tools')
from CIFAR10 import CIFAR10Trainer
from CIFAR100 import CIFAR100Trainer

### Using CIFAR10 to train a simple CNN in Chapter 3
After a time, we can see a limited accuracy compared with the performance of MNIST dataset although both the train and valid loss is very low. Because the complex dataset always need deeper network to train and test. Until 5/18/2024, the sota has 99.5% accuracy.

In [1]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, input_channels, output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # we can see fc1 and fc2 as a MLP
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, output_size)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
cnn = CNN(input_channels=3, output_size=10)
cnn_trainer_10 = CIFAR10Trainer(cnn, loss='CE', lr=0.01, optimizer='SGD', batch_size=128, epoch=30, model_type='classification')
cnn_trainer_10.train()
cnn_trainer_10.test()

Files already downloaded and verified
Files already downloaded and verified
2024-05-18 22:23:44
Epoch 1 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 38.55it/s, val_acc=53.4, val_loss=0.0103]


2024-05-18 22:23:56
Epoch 2 / 30


[Valid]: 100%|██████████████████████| 40/40 [00:01<00:00, 36.81it/s, val_acc=60.4, val_loss=0.00895]


2024-05-18 22:24:08
Epoch 3 / 30


...

[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 28.85it/s, val_acc=72.5, val_loss=0.0177]


2024-05-18 22:30:01
Epoch 29 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 30.13it/s, val_acc=72.5, val_loss=0.0178]


2024-05-18 22:30:14
Epoch 30 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 29.10it/s, val_acc=72.5, val_loss=0.0179]


### Using CIFAR100 to train a simple CNN in Chapter 3
On CIFAR100, CNN is very limited.Until 5/18/2024, the sota has 96.08% accuracy.

In [ ]:
cnn_100 = CNN(input_channels=3, output_size=100)
cnn_trainer_100 = CIFAR100Trainer(cnn_100, loss='CE', lr=0.01, optimizer='SGD', batch_size=128, epoch=30, model_type='classification')
cnn_trainer_100.train()
cnn_trainer_100.test()

Files already downloaded and verified
Files already downloaded and verified
2024-05-18 22:48:23
Epoch 1 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 35.84it/s, val_acc=16.3, val_loss=0.0286]


2024-05-18 22:48:36
Epoch 2 / 30


[Valid]: 100%|████████████████████████| 40/40 [00:01<00:00, 32.76it/s, val_acc=23.5, val_loss=0.025]


2024-05-18 22:48:48
Epoch 3 / 30


...

[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 34.31it/s, val_acc=36.1, val_loss=0.0553]


2024-05-18 22:54:41
Epoch 29 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 34.12it/s, val_acc=35.8, val_loss=0.0568]


2024-05-18 22:54:56
Epoch 30 / 30


[Valid]: 100%|██████████████████████████| 40/40 [00:01<00:00, 32.88it/s, val_acc=37, val_loss=0.058]


# Train GatedCNN on CIFAR10/100
GatedCNN has deeper network, let's what result we will get on CIFAR10/100. We use the same settings(lr, optimizer, bs and epoch) from the CNN training.

In [2]:
import sys
import torch.nn as nn
sys.path.append('../tools')
from CIFAR10 import CIFAR10Trainer
from CIFAR100 import CIFAR100Trainer

In [ ]:
model = GatedCNN(input_channels=3, output_size=10)
trainer = CIFAR10Trainer(model, loss='CE', lr=0.01, optimizer='SGD', batch_size=128, epoch=30, model_type='classification')
trainer.train()
trainer.test()

Files already downloaded and verified
Files already downloaded and verified
2024-05-18 22:57:18
Epoch 1 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 29.74it/s, val_acc=43.5, val_loss=0.0125]


2024-05-18 22:57:34
Epoch 2 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 30.63it/s, val_acc=52.6, val_loss=0.0107]


2024-05-18 22:57:50
Epoch 3 / 30


...

[Valid]: 100%|█████████████████████████| 40/40 [00:01<00:00, 26.66it/s, val_acc=70, val_loss=0.0118]


2024-05-18 23:05:16
Epoch 29 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 27.00it/s, val_acc=68.9, val_loss=0.0137]


2024-05-18 23:05:34
Epoch 30 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 26.48it/s, val_acc=70.9, val_loss=0.0131]


In [ ]:
model_100 = GatedCNN(input_channels=3, output_size=100)
trainer_100 = CIFAR100Trainer(model_100, loss='CE', lr=0.01, optimizer='SGD', batch_size=128, epoch=30, model_type='classification')
trainer_100.train()
trainer_100.test()

Files already downloaded and verified
Files already downloaded and verified
2024-05-18 23:06:43
Epoch 1 / 30


[Valid]: 100%|█████████████████████████| 40/40 [00:01<00:00, 27.86it/s, val_acc=12, val_loss=0.0302]


2024-05-18 23:07:00
Epoch 2 / 30


[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 28.56it/s, val_acc=16.9, val_loss=0.0275]


2024-05-18 23:07:16
Epoch 3 / 30


...

[Valid]: 100%|███████████████████████| 40/40 [00:01<00:00, 26.94it/s, val_acc=42.1, val_loss=0.0214]


2024-05-18 23:14:55
Epoch 29 / 30


[Valid]: 100%|████████████████████████| 40/40 [00:01<00:00, 27.49it/s, val_acc=42.1, val_loss=0.023]


2024-05-18 23:15:12
Epoch 30 / 30


[Valid]: 100%|████████████████████████| 40/40 [00:01<00:00, 26.94it/s, val_acc=41.8, val_loss=0.023]
